In [12]:
import re

html_string = '<h2>Accessibility links</h2>\n\n<h2 class="nw-c-breaking-news-banner__h2 gel-paragon-bold"><span aria-hidden="true">Breaking</span><span class="gs-u-vh">Breaking news</span></h2>\n\n<h2 class="gs-u-vh">Top Stories</h2><div class="gel-wrap gs-u-pt+"><div class="gel-layout gel-layout--no-flex nw-c-top-stories--standard nw-c-top-stories--international">'

regex = re.compile(r'<h\d(?:.*?)>(.*?)<\/h\d>')

matches = regex.findall(html_string)
print(matches)



['Accessibility links', '<span aria-hidden="true">Breaking</span><span class="gs-u-vh">Breaking news</span>', 'Top Stories']


In [16]:
import requests
from bs4 import BeautifulSoup

response = requests.get('https://www.bbc.com/news')
contents = response.text

#Match headers
regexHeder = re.compile(r'<h\d(?:.*?)>(.*?)<\/h\d>')
matches = regex.findall(contents)
print(matches)
print(len(matches))


['Accessibility links', 'News Navigation', '<span aria-hidden="true">Breaking</span><span class="gs-u-vh">Breaking news</span>', 'Top Stories', 'War in Ukraine one year on', 'Must see', 'Most watched', 'Full story', 'Most read', 'Around the BBC', 'Sport', 'Find us here', 'News Navigation', 'Explore the BBC']
